In [1]:
%matplotlib inline
import nuwinter
import matplotlib.pyplot as plt
from nuztf.neutrino_scanner import NeutrinoScanner
from astropy import units as u
from nuwinter.avro import load_avro
from nuwinter.plot import ann_fields, generate_single_page, decode_img
from nuwinter.paths import get_pdf_path
from nuwinter.utils import deduplicate_df
from pathlib import Path
import pandas as pd
import json
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib.colors import Normalize
from matplotlib.ticker import MultipleLocator
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
from tqdm import tqdm
from nuztf.skymap_scanner import SkymapScanner
from winterapi import WinterAPI
from astropy.time import Time
from shutil import unpack_archive
from nuwinter.skyportal import SkyportalClient, export_sources_to_skyportal

/opt/anaconda3/envs/nuwinter/lib/python3.11/site-packages/ligo/lw/lsctables.py:89: UserWarning: Wswiglal-redir-stdio:

SWIGLAL standard output/error redirection is enabled in IPython.
This may lead to performance penalties. To disable locally, use:

with lal.no_swig_redirect_standard_output_error():
    ...

To disable globally, use:

lal.swig_redirect_standard_output_error(False)

Note however that this will likely lead to error messages from
LAL functions being either misdirected or lost when called from
Jupyter notebooks.

To suppress this warning, use:

import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

  import lal


In [2]:
base_dir = Path.home().joinpath("Data/winter")

name = "IC250706A"
# name = "IC241006A"
program = "2024A001"

# name = "S250206dm"

prob_threshold = 0.95
n_days = 14

In [3]:
outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
outpath

'/Users/rdstein/Data/nuwinter/IC250706A/winter_candidate_pdf/combined_ztf.pdf'

In [4]:
nu = NeutrinoScanner(name) if "IC" in name else SkymapScanner(
    event=name,
    prob_threshold=prob_threshold,
    n_days=n_days
)

{'node': {'id': 'Q2lyY3VsYXI6NDEwMjE=', 'id_': '41021', 'received': '2025-07-08T09:58:11', 'subject': 'IceCube-250706A: EP-FXT follow-up observations', 'evtidCircular': {'event': 'IC 250706A'}, 'cid': 41021, 'evtid': 200276, 'oidCircular': {'telescope': None, 'detector': None, 'oidEvent': {'wavelength': None, 'messenger': None}}}}
{'node': {'id': 'Q2lyY3VsYXI6NDEwMjA=', 'id_': '41020', 'received': '2025-07-07T20:54:25', 'subject': 'Fermi-LAT Gamma-ray Observations of IceCube-250706A', 'evtidCircular': {'event': 'IC 250706A'}, 'cid': 41020, 'evtid': 200276, 'oidCircular': {'telescope': 'Fermi', 'detector': 'LAT', 'oidEvent': {'wavelength': 'HE', 'messenger': 'EM'}}}}
{'node': {'id': 'Q2lyY3VsYXI6NDEwMDM=', 'id_': '41003', 'received': '2025-07-07T08:47:45', 'subject': 'IceCube-250706A: COLIBRÍ optical upper limits', 'evtidCircular': {'event': 'IC 250706A'}, 'cid': 41003, 'evtid': 200276, 'oidCircular': {'telescope': None, 'detector': None, 'oidEvent': {'wavelength': 'optical', 'messenger

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196608/196608 [00:00<00:00, 294610.53it/s]


In [5]:
ztf_initial_cache = nu.get_initial_cache_path()

download_ztf = False

if not (ztf_initial_cache.exists()):
    download_ztf = True

if ztf_initial_cache.exists():
    if not ztf_initial_cache.stat().st_size > 2:
        download_ztf = True

if download_ztf:
    print(f"No file found at {ztf_initial_cache}")
    if isinstance(nu, SkymapScanner):
        nu.get_alerts()
        nu.filter_alerts()
    else:
        nu.scan_area(t_max=nu.t_min + 14.)

ztf_initial_cache

PosixPath('/Users/rdstein/Data/nuztf/cache/candidates/IceCube-250706A/initial_stage.json')

In [6]:
times = [nu.t_min + (i - 1) * u.day for i in range(n_days + 1)]
nights = [str(x).split("T")[0].replace("-", "") for x in times]
date = str(Time.now()).split(" ")[0].replace("-", "")
idx = nights.index(date) + 1
nights = nights[:idx]

In [7]:
# Autodownload
download = False

if download:
    winter = WinterAPI()
    res, avros = winter.query_images_by_program(program, start_date=nights[0], end_date=nights[-1], image_type="avro")

    if len(avros) ==0:
        print("No avros found. Searching for raw images instead.")
        res, exposures = winter.query_images_by_program(program, start_date=nights[0], end_date=nights[-1], image_type="raw")
        print(exposures)
        raise FileNotFoundError("No avros found on server")

    print(f"Found {len(avros)} images")
    
    for night in sorted(set(avros["nightdate"])):
        mask = avros["nightdate"] == night

        print(f"Downloading night {night} {mask.sum()}")
        
        savepaths = avros[mask]["savepath"].tolist()

        res, output_path = winter.download_image_list(program_name=program, image_type="avro", paths=savepaths, output_dir=base_dir)
        res.raise_for_status()

        # Extract and clean up zip
        extract_dir = base_dir / f"{night.replace('-', '')}/avro"
        unpack_archive(str(output_path), str(extract_dir), 'zip')
        sub_dirs = [x for x in extract_dir.glob("*") if x.is_dir()] 
        
        for sub_dir in sub_dirs:
            avro_list = sub_dir.glob("*.avro")
            for avro in avro_list:
                new_path = extract_dir / avro.name
                avro.rename(new_path)
        
            sub_dir.rmdir()
        
        output_path.unlink()

In [8]:
all_res = []

for night in sorted(nights):
    
    avro_dir = base_dir / f"{night}/avro"

    print(f"Looking for avros in {avro_dir}")
    
    avro_files = list(avro_dir.glob("*.avro"))
    
    for path in avro_files:
        all_res.append(load_avro(path))

if len(all_res) == 0:
    raise FileNotFoundError(f"No avro files found for any night, please download these first using the download notebook!")

res_winter = pd.concat(all_res)

Looking for avros in /Users/rdstein/Data/winter/20250705/avro
Looking for avros in /Users/rdstein/Data/winter/20250706/avro
Looking for avros in /Users/rdstein/Data/winter/20250707/avro
Looking for avros in /Users/rdstein/Data/winter/20250708/avro


In [9]:
in_contour = []

for _, row in res_winter.iterrows():
    in_contour.append(nu.in_contour(row["ra"], row["dec"]))

print(f"{np.sum(in_contour)} / {len(in_contour)} candidates in contour")

res_winter = res_winter[in_contour]

3435 / 11442 candidates in contour


In [10]:
res_winter

,objectid,cutout_science,cutout_template,cutout_difference,prv_candidates,candid,deprecated,jd,fid,exptime,...,tmkmag3,tmobjectid3,distgaia,plxgaia,ruwegaia,distgaiabright,plxgaiabright,ruwegaiabright,ztfname,distztf
80,WNTR25ermiv,b'\x1f\x8b\x08\x00~\xafkh\x02\xff\xec\x9cy8\x9...,b'\x1f\x8b\x08\x00~\xafkh\x02\xff\xec\x9ay4\x9...,b'\x1f\x8b\x08\x00~\xafkh\x02\xff\xec\xbci4\x9...,candid deprecated jd fid ex...,14033862,False,2.460864e+06,2,720.0,...,NaN,nan,18.671158,0.982115,1.020579,67.843422,74.054405,1.027648,nan,NaN
0,WNTR25erivx,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xec\x98iT\x8...,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xec\x97\xf9W...,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xec\x97y4\xd...,candid deprecated jd fid ex...,14031461,False,2.460864e+06,2,960.0,...,NaN,nan,18.290224,0.221262,1.022799,NaN,NaN,NaN,nan,NaN
1,WNTR25erivy,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xecwi4\x97\x...,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00=\xafkh\x02\xff\xecwg8\xd5\x...,candid deprecated jd fid ex...,14031462,False,2.460864e+06,2,960.0,...,NaN,nan,19.141127,0.492237,0.991085,NaN,NaN,NaN,nan,NaN
2,WNTR25erivz,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x97\xf7;...,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x97y4\xd...,candid deprecated jd fid ex...,14031463,False,2.460864e+06,2,960.0,...,NaN,nan,10.296278,0.221262,1.022799,NaN,NaN,NaN,nan,NaN
3,WNTR25eriwa,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x9biT\x8...,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x97\xf97...,b'\x1f\x8b\x08\x00C\xafkh\x02\xff\xec\x99\xf77...,candid deprecated jd fid ex...,14031464,False,2.460864e+06,2,960.0,...,NaN,nan,2.838943,0.132176,1.041692,60.238808,63.636875,1.722483,nan,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,WNTR25ertwq,b'\x1f\x8b\x08\x00\x13\xb0kh\x02\xff\xec\x99i4...,b'\x1f\x8b\x08\x00\x13\xb0kh\x02\xff\xec\x98\x...,b'\x1f\x8b\x08\x00\x13\xb0kh\x02\xff\xecw\xf77...,candid deprecated jd fid ex...,14038980,False,2.460864e+06,2,960.0,...,NaN,nan,24.940752,3.315387,1.042536,NaN,NaN,NaN,nan,NaN
313,WNTR25ertww,b'\x1f\x8b\x08\x00\x14\xb0kh\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00\x14\xb0kh\x02\xff\xec\x97yT...,b'\x1f\x8b\x08\x00\x14\xb0kh\x02\xff\xec\x97\x...,candid deprecated jd fid ex...,14038986,False,2.460864e+06,2,960.0,...,NaN,nan,10.036598,1.629807,0.978017,NaN,NaN,NaN,nan,NaN
319,WNTR25ertxc,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97yP...,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97\x...,candid deprecated jd fid ex...,14038992,False,2.460864e+06,2,960.0,...,NaN,nan,2.254621,1.629807,0.978017,NaN,NaN,NaN,nan,NaN
320,WNTR25ertxd,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97\x...,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97yT...,b'\x1f\x8b\x08\x00\x15\xb0kh\x02\xff\xec\x97\x...,candid deprecated jd fid ex...,14038993,False,2.460864e+06,2,960.0,...,NaN,nan,11.128734,1.629807,0.978017,NaN,NaN,NaN,nan,NaN


In [11]:
lims = []

for t in set(res_winter["jd"]):
    for b in set(res_winter["boardid"]):
        mask = (res_winter["jd"] == t) & ((res_winter["boardid"]) == b)
        match = res_winter[mask]
        if len(match) > 0:
            lims.append(match["diffmaglim"].iloc[0])

print(f"Found avros from {len(lims)} unique difference images, with median depth of {np.median(lims):.1f} AB mag")

Found avros from 21 unique difference images, with median depth of 18.8 AB mag


In [12]:
with open(ztf_initial_cache, "r") as f:
    res_ztf = json.load(f)
    
ztf_sources = []

for entry in res_ztf:

    new_dict = {}
    
    for field in ["candid", "objectId"]:
        new_dict[field] = entry[field]

    for key, val in entry["candidate"].items():
        new_dict[key] = val

    ztf_sources.append(new_dict)

ztf_df = pd.DataFrame(ztf_sources)
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,3109207433615010002,ZTF19aavbowi,2.460864e+06,1,3109207433615,21.352169,ztf_20250707206875_001723_zg_c10_o_q1_scimrefd...,Stein,2,f,...,28.760000,0.535,0.282323,0.044752,-999.000000,16.822226,-999.000000,300.0,0.984205,d6_m7
1,3109207433615010001,ZTF25aaryshi,2.460864e+06,1,3109207433615,21.352169,ztf_20250707206875_001723_zg_c10_o_q1_scimrefd...,Stein,2,f,...,28.760000,0.535,0.282323,0.104366,-999.000000,18.524956,-999.000000,300.0,0.999391,d6_m7
2,3109329423615010002,ZTF19aavbowi,2.460864e+06,2,3109329423615,21.439934,ztf_20250707329074_001723_zr_c10_o_q1_scimrefd...,Stein,2,f,...,28.858000,0.535,0.285899,0.440993,-999.000000,16.822226,-999.000000,300.0,0.991359,d6_m7
3,3109329423315015011,ZTF19aawbdwc,2.460864e+06,2,3109329423315,21.436428,ztf_20250707329074_001723_zr_c09_o_q2_scimrefd...,Stein,2,t,...,28.865999,0.536,0.279061,0.272875,-999.000000,18.272873,-999.000000,300.0,0.999972,d6_m7
4,3109329423615010001,ZTF25aaryshi,2.460864e+06,2,3109329423615,21.439934,ztf_20250707329074_001723_zr_c10_o_q1_scimrefd...,Stein,2,f,...,28.858000,0.535,0.285899,0.191576,-999.000000,18.524956,-999.000000,300.0,0.999274,d6_m7
5,3109207435015015046,ZTF25abachki,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,t,...,28.830000,0.540,0.280091,8.878161,50.984188,14.472803,12.301832,300.0,0.000033,d6_m7
6,3109207435015010012,ZTF18aamkvme,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,f,...,28.830000,0.540,0.280091,0.440306,48.958248,15.080515,12.301832,300.0,0.647073,d6_m7
7,3109207435515015042,ZTF25abachnl,2.460864e+06,1,3109207435515,21.385077,ztf_20250707206875_001723_zg_c14_o_q4_scimrefd...,Stein,2,t,...,28.799000,0.549,0.278047,17.884842,17.884842,9.609024,9.609024,300.0,0.000002,d6_m7
8,3109207435015010011,ZTF25abachkh,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,f,...,28.830000,0.540,0.280091,16.155249,-999.000000,19.647436,-999.000000,300.0,0.999985,d6_m7
9,3109329425015010004,ZTF25aakgusc,2.460864e+06,2,3109329425015,21.369543,ztf_20250707329074_001723_zr_c13_o_q3_scimrefd...,Stein,2,f,...,28.843000,0.541,0.283951,0.200373,-999.000000,20.087511,-999.000000,300.0,0.998657,d6_m7


In [13]:
ztf_df

,candid,objectId,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,isdiffpos,...,zpmed,clrmed,clrrms,neargaia,neargaiabright,maggaia,maggaiabright,exptime,drb,drbversion
0,3109207433615010002,ZTF19aavbowi,2.460864e+06,1,3109207433615,21.352169,ztf_20250707206875_001723_zg_c10_o_q1_scimrefd...,Stein,2,f,...,28.760000,0.535,0.282323,0.044752,-999.000000,16.822226,-999.000000,300.0,0.984205,d6_m7
1,3109207433615010001,ZTF25aaryshi,2.460864e+06,1,3109207433615,21.352169,ztf_20250707206875_001723_zg_c10_o_q1_scimrefd...,Stein,2,f,...,28.760000,0.535,0.282323,0.104366,-999.000000,18.524956,-999.000000,300.0,0.999391,d6_m7
2,3109329423615010002,ZTF19aavbowi,2.460864e+06,2,3109329423615,21.439934,ztf_20250707329074_001723_zr_c10_o_q1_scimrefd...,Stein,2,f,...,28.858000,0.535,0.285899,0.440993,-999.000000,16.822226,-999.000000,300.0,0.991359,d6_m7
3,3109329423315015011,ZTF19aawbdwc,2.460864e+06,2,3109329423315,21.436428,ztf_20250707329074_001723_zr_c09_o_q2_scimrefd...,Stein,2,t,...,28.865999,0.536,0.279061,0.272875,-999.000000,18.272873,-999.000000,300.0,0.999972,d6_m7
4,3109329423615010001,ZTF25aaryshi,2.460864e+06,2,3109329423615,21.439934,ztf_20250707329074_001723_zr_c10_o_q1_scimrefd...,Stein,2,f,...,28.858000,0.535,0.285899,0.191576,-999.000000,18.524956,-999.000000,300.0,0.999274,d6_m7
5,3109207435015015046,ZTF25abachki,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,t,...,28.830000,0.540,0.280091,8.878161,50.984188,14.472803,12.301832,300.0,0.000033,d6_m7
6,3109207435015010012,ZTF18aamkvme,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,f,...,28.830000,0.540,0.280091,0.440306,48.958248,15.080515,12.301832,300.0,0.647073,d6_m7
7,3109207435515015042,ZTF25abachnl,2.460864e+06,1,3109207435515,21.385077,ztf_20250707206875_001723_zg_c14_o_q4_scimrefd...,Stein,2,t,...,28.799000,0.549,0.278047,17.884842,17.884842,9.609024,9.609024,300.0,0.000002,d6_m7
8,3109207435015010011,ZTF25abachkh,2.460864e+06,1,3109207435015,21.356165,ztf_20250707206875_001723_zg_c13_o_q3_scimrefd...,Stein,2,f,...,28.830000,0.540,0.280091,16.155249,-999.000000,19.647436,-999.000000,300.0,0.999985,d6_m7
9,3109329425015010004,ZTF25aakgusc,2.460864e+06,2,3109329425015,21.369543,ztf_20250707329074_001723_zr_c13_o_q3_scimrefd...,Stein,2,f,...,28.843000,0.541,0.283951,0.200373,-999.000000,20.087511,-999.000000,300.0,0.998657,d6_m7


In [14]:
if len(ztf_df) > 0:
    ztf_positions = SkyCoord(ztf_df["ra"].to_numpy()*u.deg, ztf_df["dec"].to_numpy()*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(ztf_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    mask = d2d < crossmatch_radius
    
    cut = res_winter[mask]
    cut.reset_index(inplace=True)
    cut

In [15]:
if len(ztf_df) > 0:
    ztf_names = []

    for i in idx[mask]:
        ztf_names.append(ztf_df["objectId"].iloc[i])
    
    cut = cut.assign(ztf_name=ztf_names)
    if len(cut) > 0:
        cut = deduplicate_df(cut)
    cut
else:
    cut = pd.DataFrame()

In [16]:
if len(cut) > 0:
    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf")
    
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields)
            pdf.savefig()
            plt.close()

    outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf_crossmatched")

    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut.iterrows(), total=len(cut)):
            generate_single_page(row, ann_fields=["ztf_name"] + ann_fields, crossmatch=True)
            pdf.savefig()
            plt.close()

    export_sources_to_skyportal(cut)
else:
    print("No candidates found via ZTF crossmatch")

No candidates found via ZTF crossmatch


In [17]:
len(res_winter["objectid"])

3435

In [18]:
# crossmatch_radius = 2.0 * u.arcsec

# winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')


# match = []

# for i, c in enumerate(winter_positions):
#     if i < len(winter_positions) - 1:
#         idx, d2d, _ = c.match_to_catalog_sky(winter_positions[i+1:])
#         if d2d < crossmatch_radius:
#             match.append(idx)
#         else:
#             match.append(None)
#     else:
#         match.append(None)

In [19]:
# res_winter["match"] = match
# res_winter_match = res_winter[pd.notnull(match)]
res_winter_match = res_winter

In [20]:
# all_matched = []

# for i, row in res_winter_match.iterrows():
#     c = SkyCoord(row["ra"]*u.deg, row["dec"]*u.deg, frame='icrs')
#     sep = c.separation(winter_positions)
#     mask = (sep.arcsec > 0.) & (sep.arcsec < crossmatch_radius.value)
#     matches = res_winter.loc[mask]
#     row["ndethist"] = mask.sum()
#     row["prv_candidates"] = res_winter.loc[mask]
#     print(row["objectid"], res_winter.loc[mask]["objectid"])
#     all_matched.append(row)

# if len(all_matched) > 0:
#     res_winter_match = pd.concat(all_matched, axis=1).transpose()
#     res_winter_match

In [21]:
sgscore_cut = 0.5
dist_cut_arcsec = 5.

star_cut = (pd.notnull(res_winter_match["distpsnr1"]) & (res_winter_match["distpsnr1"] < dist_cut_arcsec) & pd.notnull(res_winter_match["sgscore1"]) & (res_winter_match["sgscore1"] > sgscore_cut))

masks = [
    res_winter_match["ndethist"] > 0,
    ~star_cut,
    # res_winter_match["nbad"] < 2,
    # res_winter_match["chipsf"] < 3.0,
    # res_winter_match["sumrat"] > 0.7,
    # res_winter_match["fwhm"] < 10.0,
    # res_winter_match["magdiff"] < 1.6,
    # res_winter_match["magdiff"] > -1.0,
    # res_winter_match["mindtoedge"] > 50.0,
]

mask = np.ones(len(res_winter_match), dtype=bool)
for new_mask in masks:
    mask *= new_mask
    
print(f"{np.sum(mask)} / {len(mask)} detections pass all cuts")
cut2 = res_winter_match[mask]

if len(cut2) > 0:
    cut2 = deduplicate_df(cut2)
    cut2.sort_values(by="objectid", inplace=True)
    
    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=False)
            pdf.savefig()
            plt.close()

    outpath = str(get_pdf_path(name)).replace("combined", "combined_2dets_crossmatched")
    print(f"Saving PDF to {outpath}")
    
    with PdfPages(outpath) as pdf:
        for i, row in tqdm(cut2.iterrows(), total=len(cut2)):
            generate_single_page(row, ann_fields=ann_fields, crossmatch=True)
            pdf.savefig()
            plt.close()

    export_sources_to_skyportal(cut2)

32 / 3435 detections pass all cuts
Saving PDF to /Users/rdstein/Data/nuwinter/IC250706A/winter_candidate_pdf/combined_2dets.pdf


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 24.91it/s]


Saving PDF to /Users/rdstein/Data/nuwinter/IC250706A/winter_candidate_pdf/combined_2dets_crossmatched.pdf


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:11<00:00,  1.63it/s]


In [22]:
list(res_winter_match.columns)

['objectid',
 'cutout_science',
 'cutout_template',
 'cutout_difference',
 'prv_candidates',
 'candid',
 'deprecated',
 'jd',
 'fid',
 'exptime',
 'ndethist',
 'jdstarthist',
 'jdendhist',
 'progname',
 'programid',
 'isdiffpos',
 'fieldid',
 'boardid',
 'ra',
 'dec',
 'magzpsci',
 'magzpsciunc',
 'magzpscirms',
 'diffmaglim',
 'magpsf',
 'sigmapsf',
 'chipsf',
 'magap',
 'sigmagap',
 'magapbig',
 'sigmagapbig',
 'magdiff',
 'magfromlim',
 'distnr',
 'magnr',
 'sigmagnr',
 'xpos',
 'ypos',
 'sky',
 'fwhm',
 'classtar',
 'mindtoedge',
 'seeratio',
 'aimage',
 'bimage',
 'aimagerat',
 'bimagerat',
 'elong',
 'nneg',
 'nbad',
 'sumrat',
 'dsnrms',
 'ssnrms',
 'dsdiff',
 'scorr',
 'clrcoeff',
 'clrcounc',
 'zpclrcov',
 'zpmed',
 'clrmed',
 'clrrms',
 'rb',
 'rbversion',
 'ssdistnr',
 'ssmagnr',
 'ssnamenr',
 'tooflag',
 'nmtchps',
 'psra1',
 'psdec1',
 'psobjectid1',
 'sgmag1',
 'srmag1',
 'simag1',
 'szmag1',
 'sgscore1',
 'distpsnr1',
 'ps1class1',
 'ps1probstar1',
 'ps1probqso1',
 'ps1p

In [23]:
# Download the TNS after JSON for GW event, 

tns_dir = get_pdf_path(name).parent.parent

print(f"Looking for TNS json in {tns_dir}")

tns_blob = list(tns_dir.glob("*.json"))

tns_df = pd.read_json(tns_blob[0]).transpose() if len(tns_blob) > 0 else pd.DataFrame()
tns_df

Looking for TNS json in /Users/rdstein/Data/nuwinter/IC250706A


""


In [24]:
if len(tns_df) > 0:
    tns_positions = SkyCoord(tns_df["ra"].to_numpy(dtype=float)*u.deg, tns_df["declination"].to_numpy(dtype=float)*u.deg, frame='icrs')
    
    winter_positions = SkyCoord(res_winter["ra"].to_numpy()*u.deg, res_winter["dec"].to_numpy()*u.deg, frame='icrs')
    idx, d2d, _ = winter_positions.match_to_catalog_sky(tns_positions)
    
    crossmatch_radius = 3.0 * u.arcsec
    mask = d2d < crossmatch_radius

    print(f"Found {mask.sum()}/{len(winter_positions)} matches to {len(tns_df)} TNS sources.")

    cut_tns = res_winter.copy()

In [25]:
if len(tns_df) > 0:
    cut_tns

In [26]:
if len(tns_df) > 0:
    if len(cut_tns) > 0:
        outpath = str(get_pdf_path(name)).replace("combined", "combined_tns")
        
        print(f"Saving PDF to {outpath}")
        
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_tns.iterrows(), total=len(cut)):
                generate_single_page(row, ann_fields=["ztf_name"] + ann_fields)
                pdf.savefig()
                plt.close()
    
        outpath = str(get_pdf_path(name)).replace("combined", "combined_ztf_crossmatched")
    
        with PdfPages(outpath) as pdf:
            for i, row in tqdm(cut_tns.iterrows(), total=len(cut)):
                generate_single_page(row, ann_fields=["ztf_name"] + ann_fields, crossmatch=True)
                pdf.savefig()
                plt.close()

    export_sources_to_skyportal(tns_df)
else:
    print("No candidates found via TNS")

No candidates found via TNS
